In [1]:
import requests 
from bs4 import BeautifulSoup
from pprint import pprint
from tqdm import tqdm
import traceback

In [1]:
import os 
import json

To Crawl OSHO Discourses we have to go to individual pages from a-z.

    -Each page under that character has a theme of discourses starting from that character

    -Every theme or discourse topic name will have episodes
 
    -We first find the full hierarchy of episodes

In [2]:
all_chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 
             'u', 'v', 'w', 'x', 'y', 'z']

In [3]:
base_link = "https://oshoworld.com/osho-audio-discourse-english-a/"

### For every charcater we have a seperate Link

In [5]:
all_links = [base_link[:-2] + char + base_link[-1] for char in all_chars]
pprint(all_links, indent=10)

[         'https://oshoworld.com/osho-audio-discourse-english-a/',
          'https://oshoworld.com/osho-audio-discourse-english-b/',
          'https://oshoworld.com/osho-audio-discourse-english-c/',
          'https://oshoworld.com/osho-audio-discourse-english-d/',
          'https://oshoworld.com/osho-audio-discourse-english-e/',
          'https://oshoworld.com/osho-audio-discourse-english-f/',
          'https://oshoworld.com/osho-audio-discourse-english-g/',
          'https://oshoworld.com/osho-audio-discourse-english-h/',
          'https://oshoworld.com/osho-audio-discourse-english-i/',
          'https://oshoworld.com/osho-audio-discourse-english-j/',
          'https://oshoworld.com/osho-audio-discourse-english-k/',
          'https://oshoworld.com/osho-audio-discourse-english-l/',
          'https://oshoworld.com/osho-audio-discourse-english-m/',
          'https://oshoworld.com/osho-audio-discourse-english-n/',
          'https://oshoworld.com/osho-audio-discourse-english-

____________________________________________________________________________________
### From Table extract individual episodes and Links
    -2nd column is having topic name and Link

In [6]:
def find_everything_in_sub_tables(table):
    all_cells = {}
    all_rows = table.find_all("tr")
    
    for each_row in all_rows:
        all_td = each_row.find_all("td")
        topic_name = all_td[1].get_text()
        
        link = ""
        link_td = all_td[1]
        
        anchor_tag = link_td.find_all("a")
        
        if len(anchor_tag):
            if "href" in anchor_tag[0].attrs:
                link = anchor_tag[0].get("href")
        
        all_cells[topic_name] = link
            
    return all_cells

#### For Every Character we find all the topics correspoding to it
    -Fetch every Row
    -second column corresponds to topic name
    -third column is the Hyperlink

In [7]:
def find_everything_in_character_tables(table):
    all_cells = {}
    all_rows = table.find_all("tr")
    
    for each_row in all_rows:
        all_td = each_row.find_all("td")
        topic_name = all_td[1].get_text()
        
        link = ""
        link_td = all_td[2]
        
        anchor_tag = link_td.find_all("a")
        
        if len(anchor_tag):
            if "href" in anchor_tag[0].attrs:
                link = anchor_tag[0].get("href")
        
        all_cells[topic_name] = link
            
    return all_cells

#### Crawl each Topic
    -Go to each Topic
    -Find all the episodes in it

In [8]:
def extract_sub_url(sub_json):
    full_json = []
    unable_to_crawl_topics = []
    
    for k,v in tqdm(sub_json.items()):
        sub_url = v
        sub_page_json = {}
        j = {}

        try:
            print(sub_url)
            res = requests.get(sub_url)

            html_page = res.content
            soup = BeautifulSoup(html_page, 'html.parser')

            all_tables = soup.find_all("table")
            current_table = all_tables[-1]
            j = find_everything_in_sub_tables(current_table)

        except:
            traceback.print_exc()
            pprint("Unable to Crawl TOPIC Link \n {}".format(sub_url))
            unable_to_crawl_topics.append(v)

        sub_page_json[k] = j
        full_json.append(sub_page_json)
    
    return full_json, unable_to_crawl_topics

### Orchestration Code
#### -- Crawls each charcater Link
#### -- Stores the final URL to be crawled where each individual OSHO text is written

In [9]:
full_jsons = {}
unable_to_crawl_characters_url = []
unable_to_crawl_topics = []

# Going through all the URL's corresponding to each character
for current_url in tqdm(all_links):
    this_url_json = []
    
    try:
        pprint(current_url)
        res = requests.get(current_url)
        
        html_page = res.content
        soup = BeautifulSoup(html_page, 'html.parser')
        
        all_tables = soup.find_all("table")
        current_table = all_tables[-1]
        
        this_url_json = find_everything_in_character_tables(current_table)
        this_url_json,unable_to_crawl_topic_urls  = extract_sub_url(this_url_json)
        
        unable_to_crawl_topics.append(unable_to_crawl_topic_urls)
    except:
        traceback.print_exc()
        pprint("Unable to Crawl Character LINK \n {}".format(current_url))
        unable_to_crawl_characters_url.append(current_url)
    
    full_jsons[current_url] = this_url_json

  0%|                                                                                           | 0/26 [00:00<?, ?it/s]

'https://oshoworld.com/osho-audio-discourse-english-a/'



  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

https://oshoworld.com/osho-the-art-of-dying-by-osho-01/



 17%|██████████████                                                                      | 1/6 [00:14<01:11, 14.34s/it]

...


 83%|████████████████████████████████████████████████████████████████████▎             | 10/12 [00:28<00:05,  2.94s/it]

https://oshoworld.com/zarathustra-a-god-that-can-by-osho-01-23/



 92%|███████████████████████████████████████████████████████████████████████████▏      | 11/12 [00:32<00:03,  3.35s/it]

https://oshoworld.com/zarathustra-laughing-prophet-by-osho-01-23/



100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [21:17<00:00, 49.12s/it]


In [10]:
pprint(full_jsons)

{'https://oshoworld.com/osho-audio-discourse-english-a/': [{'Art of Dying 01-10Category: JEWISH MYSTICS': {' OSHO-The Art of Dying 01   (Read, Listen & Download)  ': 'https://oshoworld.com/osho-the-art-of-dying-01/',
                                                                                                           ' OSHO-The Art of Dying 02   (Read, Listen & Download)  ': 'https://oshoworld.com/osho-the-art-of-dying-02/',
                                                                                                           ' OSHO-The Art of Dying 03   (Read, Listen & Download)  ': 'https://oshoworld.com/osho-the-art-of-dying-03/',
                                                                                                           ' OSHO-The Art of Dying 04   (Read, Listen & Download)  ': 'https://oshoworld.com/osho-the-art-of-dying-04/',
                                                                                                           ' OSHO-The Art of Dying 0

                                                                                                                            ' From the False to the Truth 23   (Read, Listen & Download)  ': 'https://oshoworld.com/from-the-false-to-the-truth-23/',
                                                                                                                            ' From the False to the Truth 24   (Read, Listen & Download)  ': 'https://oshoworld.com/from-the-false-to-the-truth-24/',
                                                                                                                            ' From the False to the Truth 25   (Read, Listen & Download)  ': 'https://oshoworld.com/from-the-false-to-the-truth-25/',
                                                                                                                            ' From the False to the Truth 26   (Read, Listen & Download)  ': 'https://oshoworld.com/from-the-false-to-the-truth-26/',
                

                                                                                                                             ' The Language of Existence 07   (Read, Listen & Download)  ': 'https://oshoworld.com/the-language-of-existence-07/',
                                                                                                                             ' The Language of Existence 08   (Read, Listen & Download)  ': 'https://oshoworld.com/the-language-of-existence-08/',
                                                                                                                             ' The Language of Existence 09   (Read, Listen & Download)  ': 'https://oshoworld.com/the-language-of-existence-09/'}},
                                                           {'Live Zen 01-17Category: ZEN AND ZEN MASTERS': {' Live Zen 01   (Read, Listen & Download)  ': 'https://oshoworld.com/live-zen-01/',
                                                                             

                                                                                                               ' The Perfect Master Vol 1 08   (Read, Listen & Download)  ': 'https://oshoworld.com/the-perfect-master-vol-1-08/',
                                                                                                               ' The Perfect Master Vol 1 09   (Read, Listen & Download)  ': 'https://oshoworld.com/the-perfect-master-vol-1-09/',
                                                                                                               ' The Perfect Master Vol 1 10   (Read, Listen & Download)  ': 'https://oshoworld.com/the-perfect-master-vol-1-10/'}},
                                                           {'The Perfect Master Vol 2 01-10Category: SUFISM': {' The Perfect Master Vol 1 01   (Read, Listen & Download)  ': 'https://oshoworld.com/the-perfect-master-vol-1-01/',
                                                                                          

                                                                                                    ' Until You Die 05   (Listen & Download)  ': 'https://oshoworld.com/until-you-die-05/',
                                                                                                    ' Until You Die 06   (Listen & Download)  ': 'https://oshoworld.com/until-you-die-06/',
                                                                                                    ' Until You Die 07   (Listen & Download)  ': 'https://oshoworld.com/until-you-die-07/',
                                                                                                    ' Until You Die 08   (Listen & Download)  ': 'https://oshoworld.com/until-you-die-08/',
                                                                                                    ' Until You Die 09   (Listen & Download)  ': 'https://oshoworld.com/until-you-die-09/',
                                                            

In [11]:
pprint(unable_to_crawl_characters_url)

['https://oshoworld.com/osho-audio-discourse-english-q/',
 'https://oshoworld.com/osho-audio-discourse-english-x/']


In [12]:
pprint(unable_to_crawl_topics)

[['https://oshoworld.com/a-bird-on-the-wing-01-11/'],
 [],
 [],
 ['https://oshoworld.com/the-dhammapada-vol-7-01-10/'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['https://oshoworld.com/press-conference-01-11/'],
 ['https://oshoworld.com/rinzai-master-of-the-irrational-01-08/'],
 ['https://oshoworld.com/socrates-poisoned-again-01-28/'],
 [],
 [],
 [],
 [],
 [],
 []]


#### Store the Meta Information about all the links in JSONS, so that they can be used later at any time

    -- data.json has all information about the final discourse
        -- which index, topic and episode it belongs to
        
    -- index.json contains all discourses URL's only. 
    
    -- Total 2974 discourses links are there 

In [27]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(full_jsons, f, ensure_ascii=False, indent=4)

In [28]:
how_many_links = []
for char_page, j in full_jsons.items():
    for each in j:
        for k1, v1 in each.items():
            print(k1)
            for k2, v2 in v1.items():
                how_many_links.append(v2)

Art of Dying 01-10Category: JEWISH MYSTICS
Ah This 01-08Category: ZEN AND ZEN MASTERS
Ancient Music in the Pines 01-09Category: ZEN AND ZEN MASTERS
And The Flowers Showered 01-11Category: ZEN AND ZEN MASTERS
A Bird on the Wing 01-11Category: ZEN AND ZEN MASTERS
A Sudden Clash of Thunder 01-10Category: ZEN AND ZEN MASTERS
Bodhidharma 01-20Category: ZEN AND ZEN MASTERS
Buddha Emptiness of Heart 01-08Category: ZEN AND ZEN MASTERS
Be Still and Know 01-10Category: RESPONSES TO QUESTIONS
Beyond Enlightenment 01-32Category: THE WORLD TOUR
Beyond Psychology 01-44Category: THE WORLD TOUR
The Beloved Vol 1 01-10Category: BAUL MYSTICS
The Beloved Vol 2 01-10Category: BAUL MYSTICS
The Book of Wisdom Vol 1 01-16Category: BUDDHA AND BUDDHIST MASTERS
The Book of Wisdom Vol 2 01-12Category: BUDDHA AND BUDDHIST MASTERS
Come Follow Yourself Vol 01 01-10Category: JESUS
Come Follow Yourself Vol 02 01-11Category: JESUS
Come Follow Yourself Vol 03 01-10Category: JESUS
Come Follow Yourself Vol 04 01-11Catego

In [29]:
len(how_many_links)

2974

In [30]:
pprint(how_many_links)

['https://oshoworld.com/osho-the-art-of-dying-01/',
 'https://oshoworld.com/osho-the-art-of-dying-02/',
 'https://oshoworld.com/osho-the-art-of-dying-03/',
 'https://oshoworld.com/osho-the-art-of-dying-04/',
 'https://oshoworld.com/osho-the-art-of-dying-05/',
 'https://oshoworld.com/osho-the-art-of-dying-06/',
 'https://oshoworld.com/osho-the-art-of-dying-07/',
 'https://oshoworld.com/osho-the-art-of-dying-08/',
 'https://oshoworld.com/osho-the-art-of-dying-09/',
 'https://oshoworld.com/osho-the-art-of-dying-10/',
 'https://oshoworld.com/ah-this-01/',
 'https://oshoworld.com/ah-this-02/',
 'https://oshoworld.com/ah-this-03/',
 'https://oshoworld.com/ah-this-04/',
 'https://oshoworld.com/ah-this-05/',
 'https://oshoworld.com/ah-this-06/',
 'https://oshoworld.com/ah-this-07/',
 'https://oshoworld.com/ah-this-08/',
 'https://oshoworld.com/ancient-music-in-the-pines-01/',
 'https://oshoworld.com/ancient-music-in-the-pines-02/',
 'https://oshoworld.com/ancient-music-in-the-pines-03/',
 'htt

 'https://oshoworld.com/om-mani-padme-hum-07/',
 'https://oshoworld.com/om-mani-padme-hum-08/',
 'https://oshoworld.com/om-mani-padme-hum-09/',
 'https://oshoworld.com/om-mani-padme-hum-10/',
 'https://oshoworld.com/om-mani-padme-hum-11/',
 'https://oshoworld.com/om-mani-padme-hum-12/',
 'https://oshoworld.com/om-mani-padme-hum-13/',
 'https://oshoworld.com/om-mani-padme-hum-14/',
 'https://oshoworld.com/om-mani-padme-hum-15/',
 'https://oshoworld.com/om-mani-padme-hum-16/',
 'https://oshoworld.com/om-mani-padme-hum-17/',
 'https://oshoworld.com/om-mani-padme-hum-18/',
 'https://oshoworld.com/om-mani-padme-hum-19/',
 'https://oshoworld.com/om-mani-padme-hum-20/',
 'https://oshoworld.com/om-mani-padme-hum-21/',
 'https://oshoworld.com/om-mani-padme-hum-22/',
 'https://oshoworld.com/om-mani-padme-hum-23/',
 'https://oshoworld.com/om-mani-padme-hum-24/',
 'https://oshoworld.com/om-mani-padme-hum-25/',
 'https://oshoworld.com/om-mani-padme-hum-26/',
 'https://oshoworld.com/om-mani-padme-hu

In [31]:
index_url = {}
cur = 1
for link in how_many_links:
    index_url[cur] = link
    cur += 1 

In [32]:
pprint(index_url)

{1: 'https://oshoworld.com/osho-the-art-of-dying-01/',
 2: 'https://oshoworld.com/osho-the-art-of-dying-02/',
 3: 'https://oshoworld.com/osho-the-art-of-dying-03/',
 4: 'https://oshoworld.com/osho-the-art-of-dying-04/',
 5: 'https://oshoworld.com/osho-the-art-of-dying-05/',
 6: 'https://oshoworld.com/osho-the-art-of-dying-06/',
 7: 'https://oshoworld.com/osho-the-art-of-dying-07/',
 8: 'https://oshoworld.com/osho-the-art-of-dying-08/',
 9: 'https://oshoworld.com/osho-the-art-of-dying-09/',
 10: 'https://oshoworld.com/osho-the-art-of-dying-10/',
 11: 'https://oshoworld.com/ah-this-01/',
 12: 'https://oshoworld.com/ah-this-02/',
 13: 'https://oshoworld.com/ah-this-03/',
 14: 'https://oshoworld.com/ah-this-04/',
 15: 'https://oshoworld.com/ah-this-05/',
 16: 'https://oshoworld.com/ah-this-06/',
 17: 'https://oshoworld.com/ah-this-07/',
 18: 'https://oshoworld.com/ah-this-08/',
 19: 'https://oshoworld.com/ancient-music-in-the-pines-01/',
 20: 'https://oshoworld.com/ancient-music-in-the-pin

 853: 'https://oshoworld.com/the-golden-future-26/',
 854: 'https://oshoworld.com/the-golden-future-27/',
 855: 'https://oshoworld.com/the-golden-future-28/',
 856: 'https://oshoworld.com/the-golden-future-29/',
 857: 'https://oshoworld.com/the-golden-future-30/',
 858: 'https://oshoworld.com/the-golden-future-31/',
 859: 'https://oshoworld.com/the-golden-future-32/',
 860: 'https://oshoworld.com/the-golden-future-33/',
 861: 'https://oshoworld.com/the-golden-future-34/',
 862: 'https://oshoworld.com/the-golden-future-35/',
 863: 'https://oshoworld.com/the-golden-future-36/',
 864: 'https://oshoworld.com/the-golden-future-37/',
 865: 'https://oshoworld.com/the-golden-future-38/',
 866: 'https://oshoworld.com/the-golden-future-39/',
 867: 'https://oshoworld.com/the-golden-future-40/',
 868: 'https://oshoworld.com/god-is-dead-now-zen-is-01/',
 869: 'https://oshoworld.com/god-is-dead-now-zen-is-02/',
 870: 'https://oshoworld.com/god-is-dead-now-zen-is-03/',
 871: 'https://oshoworld.com/go

 1715: 'https://oshoworld.com/the-path-of-the-mystic-39/',
 1716: 'https://oshoworld.com/the-path-of-the-mystic-40/',
 1717: 'https://oshoworld.com/the-path-of-the-mystic-41/',
 1718: 'https://oshoworld.com/the-path-of-the-mystic-42/',
 1719: 'https://oshoworld.com/the-path-of-the-mystic-43/',
 1720: 'https://oshoworld.com/the-path-of-the-mystic-44/',
 1721: 'https://oshoworld.com/philosophia-ultima-01/',
 1722: 'https://oshoworld.com/philosophia-ultima-02/',
 1723: 'https://oshoworld.com/philosophia-ultima-03/',
 1724: 'https://oshoworld.com/philosophia-ultima-04/',
 1725: 'https://oshoworld.com/philosophia-ultima-05/',
 1726: 'https://oshoworld.com/philosophia-ultima-06/',
 1727: 'https://oshoworld.com/philosophia-ultima-07/',
 1728: 'https://oshoworld.com/philosophia-ultima-08/',
 1729: 'https://oshoworld.com/philosophia-ultima-09/',
 1730: 'https://oshoworld.com/philosophia-ultima-10/',
 1731: 'https://oshoworld.com/philosophia-ultima-11/',
 1732: 'https://oshoworld.com/philosophia

 2554: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-10/',
 2555: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-11/',
 2556: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-12/',
 2557: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-13/',
 2558: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-14/',
 2559: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-15/',
 2560: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-16/',
 2561: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-17/',
 2562: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-18/',
 2563: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-19/',
 2564: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-20/',
 2565: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-21/',
 2566: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-22/',
 2567: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-23/',
 2568: 'https://oshoworld.com/vigyan-bhairav-tantra-vol-2-24/',
 2569: 'https://oshoworld.com/vigyan-bha

In [33]:
with open('index.json', 'w', encoding='utf-8') as f:
    json.dump(index_url, f, ensure_ascii=False, indent=4)

In [7]:
f = open('index.json')
index_url = json.load(f)

In [8]:
cwd = os.getcwd()
cwd

'D:\\jupyter-notebooks\\imp_functions\\Tensor-Flow\\OSHO_Dataset'

In [10]:
base_dir = "files_new_1"
final_dir = os.path.join(cwd, base_dir)
final_dir

'D:\\jupyter-notebooks\\imp_functions\\Tensor-Flow\\OSHO_Dataset\\files_new_1'

In [11]:
no_text_urls = []
not_able_to_open_urls = []

for index, url in tqdm(index_url.items()):
    print(url)
    try:
        res = requests.get(url)
        html_page = res.content
        soup = BeautifulSoup(html_page, 'html.parser')

        for data in soup.find_all("div"):
            if "class" in data.attrs:
                class_name = data.get("class")
                if class_name == ['vc_acf', 'vc_txt_align_left', 'field_6402fc08169a2']:
                    cur_file_name = os.path.join(final_dir, str(index) + ".txt")
                    str_1 = data.get_text()
                    if len(str_1) > 6:
                        with open(cur_file_name, 'w', encoding='utf-8') as f:
                            f.write(str_1)
                    else:
                        no_text_urls.append(url)                     
    except:
        not_able_to_open_urls.append(url)
        traceback.print_exc()

  0%|                                                                                         | 0/2974 [00:00<?, ?it/s]

https://oshoworld.com/osho-the-art-of-dying-01/


  0%|                                                                               | 1/2974 [00:03<2:35:43,  3.14s/it]

https://oshoworld.com/osho-the-art-of-dying-02/


  0%|                                                                               | 2/2974 [00:05<2:13:06,  2.69s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▉| 2972/2974 [2:17:49<00:04,  2.16s/it]

https://oshoworld.com/zarathustra-laughing-prophet-22/


100%|███████████████████████████████████████████████████████████████████████████▉| 2973/2974 [2:17:51<00:02,  2.13s/it]

https://oshoworld.com/zarathustra-laughing-prophet-23/


100%|████████████████████████████████████████████████████████████████████████████| 2974/2974 [2:17:53<00:00,  2.78s/it]


In [13]:
print(len(no_text_urls))

954


These Links do not contain text they mostly contain the audio file of the discourse

In [14]:
pprint(no_text_urls)

['https://oshoworld.com/a-sudden-clash-of-thunder-01/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-02/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-03/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-04/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-05/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-06/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-07/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-08/',
 'https://oshoworld.com/a-sudden-clash-of-thunder-10/',
 'https://oshoworld.com/bodhidharma-12-3/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-01/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-02/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-03/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-04/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-05/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-06/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-07/',
 'https://oshoworld.com/the-book-of-wisdom-vol-2-08/',
 'https://os